### custom breakdown file for a model as a netcdf

-air sea C flux
-air sea SO flux
-air sea export
-air sea annual surface DIC
-air sea annual S (surface)
-the above two at 100m and for SO



In [6]:
import xarray as xr
import time
import glob
import re
import numpy as np


In [7]:
tr = 'TOM12_TJ_1AS6'
baseDir = '/gpfs/afm/greenocean/software/runs/'

## helper functions - general and averaging

In [8]:
def make_yearlist(yrst, yrend, dtype, tr, baseDir):
    yrs = np.arange(yrst,yrend+1,1)
    ylist = []
    for i in range(0,len(yrs)):
        ty = f'{baseDir}/{tr}/ORCA2_1m_{yrs[i]}*{dtype}*.nc'
        t2 = glob.glob(ty)
        #print(t2)
        ylist.append(t2[0])
    return ylist

def max_min_yrs(tr, baseDir):
    '''manually find the start and end year of a list of model output 
    for a given run, assuming that ptrc is being output
    needs tr - run name
    baseDir - run directory
    '''
    
    w = glob.glob(f'{baseDir}{tr}/ORCA2_1m_*ptrc*.nc')
    yrs = []
    for i in range(0,len(w)):
        ts = (w[i])
        pattern = f'{baseDir}{tr}/ORCA2_1m_'
        mod_string = re.sub(pattern, '', ts)
        yrs.append(int(mod_string[0:4]))
        
    tmin = min(yrs); tmax = max(yrs)
    return tmin, tmax


def weighted_temporal_mean(tvar):
    """
    weight by days in each month - get a yearly mean value for a quantity
    https://ncar.github.io/esds/posts/2021/yearly-averages-xarray/
    original had stuff about nans, we don't
    """
    # Determine the month length
    month_length = tvar.time_counter.dt.days_in_month

    # Calculate the weights
    wgts = month_length.groupby("time_centered.year") / month_length.groupby("time_centered.year").sum()

    # Make sure the weights in each year add up to 1
    np.testing.assert_allclose(wgts.groupby("time_centered.year").sum(xr.ALL_DIMS), 1.0)

    # Subset our dataset for our variable
    obs = tvar

    # Calculate the numerator annual
    obs_sum = (obs * wgts).resample(time_counter="A").sum(dim="time_counter")

    return obs_sum 

def masked_average(xa:xr.DataArray,
                   dim=None,
                   weights:xr.DataArray=None,
                   mask:xr.DataArray=None):
    """
    This function will average
    :param xa: dataArray
    :param dim: dimension or list of dimensions. e.g. 'lat' or ['lat','lon','time']
    :param weights: weights (as xarray)
    :param mask: mask (as xarray), True where values to be masked.
    :return: masked average xarray
    """
    #lest make a copy of the xa
    xa_copy:xr.DataArray = xa.copy()

    if mask is not None:
        xa_weighted_average = __weighted_average_with_mask(
            dim, mask, weights, xa, xa_copy
        )
    elif weights is not None:
        xa_weighted_average = __weighted_average(
            dim, weights, xa, xa_copy
        )
    else:
        xa_weighted_average =  xa.mean(dim)

    return xa_weighted_average



    # %% [markdown]
def __weighted_average(dim, weights, xa, xa_copy):
    '''helper function for masked_average'''
    _, weights_all_dims = xr.broadcast(xa, weights)  # broadcast to all dims
    x_times_w = xa_copy * weights_all_dims
    xw_sum = x_times_w.sum(dim)
    x_tot = weights_all_dims.where(xa_copy.notnull()).sum(dim=dim)
    xa_weighted_average = xw_sum / x_tot
    return xa_weighted_average


def __weighted_average_with_mask(dim, mask, weights, xa, xa_copy):
    '''helper function for masked_average'''
    _, mask_all_dims = xr.broadcast(xa, mask)  # broadcast to all dims
    xa_copy = xa_copy.where(np.logical_not(mask))
    if weights is not None:
        _, weights_all_dims = xr.broadcast(xa, weights)  # broadcast to all dims
        weights_all_dims = weights_all_dims.where(~mask_all_dims)
        x_times_w = xa_copy * weights_all_dims
        xw_sum = x_times_w.sum(dim=dim)
        x_tot = weights_all_dims.where(xa_copy.notnull()).sum(dim=dim)
        xa_weighted_average = xw_sum / x_tot
    else:
        xa_weighted_average = xa_copy.mean(dim)
    return xa_weighted_average


# ## Application 1: Weigted global average:
# Grid cells have different area, so when we do the global average, they have to be weigted by the area of each grid cell.
# Here we do it for 2 m temperature:

### extractor functions

In [9]:


# def get_pco2(t_ds, verbose):

#     tmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/regrid/mesh_mask3_6.nc')
#     tmesh['csize'] = tmesh.tmask[0,0,:,:] * tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:]

        
#     return pco2_uatm_yr, pco2_uatm_yr_so

# def get_ppt(t_ds, verbose):
#     tmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/regrid/mesh_mask3_6.nc')
#     tmesh['csize'] = tmesh.tmask[0,:,:,:] * tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:] * tmesh.e3t_0[0,:,:,:]
#     csize = tmesh['csize'].values #m3 
#     ppt = t_ds['PPT']
#     ppt_wt = (weighted_temporal_mean(ppt)).values # mol/m3/s, yearly average
    
#     yrly_ppt = np.zeros_like(ppt_wt[:,0,0,0])
#     yrly_ppt_so = np.zeros_like(ppt_wt[:,0,0,0])
    
#     for i in range(0,len(yrly_ppt)):
#         yrly_ppt[i] = np.nansum(ppt_wt[i,:,:,:]*csize[:,:,:]) #still in average of mol/s
#         yrly_ppt_so[i] = np.nansum(ppt_wt[i,:,0:37,:]*csize[:,0:37,:])

#     siy = 60*60*24*365 #seconds in year
#     pg_in_mol = 12 * 1e-15 #petagrams in a mol

#     ppt_pg_yr = yrly_ppt*siy*pg_in_mol
#     ppt_pg_yr_so = yrly_ppt_so*siy*pg_in_mol
    
#     if verbose:
#         print(ppt_pg_yr)
#         print(ppt_pg_yr_so)

#     return ppt_pg_yr, ppt_pg_yr_so

In [19]:
def get_cflx(tmin,tmax,tr, baseDir, sdir, fnam, verbose = False):
    '''take an mfdataset, calculate ts of cflx 
    for whole ocean and southern ocean, return two np arrays'''
    t_yearlist = make_yearlist(tmin,tmax,'diad',tr, baseDir)
    yrs = np.arange(tmin, tmax+1,1)
    t_ds = xr.open_mfdataset(t_yearlist)    
    #cflx is in mol/m2/s, multiply by m2 in meshmask to get mol/s/grid cell
    tmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/regrid/mesh_mask3_6.nc')
    tmesh['csize'] = tmesh.tmask[0,0,:,:] * tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:]

    cflx_mol_s = t_ds['Cflx']*tmesh['csize']
    #get a yearly mean in mol_s_gridcell- takes monthly data, gets out yearly data
    t_yearly = weighted_temporal_mean(cflx_mol_s)
    siy = 60*60*24*365 #seconds in year
    pg_in_mol = 12 * 1e-15 #petagrams in a mol
    cflx_pg_yr = (np.nansum(np.nansum(t_yearly*siy*pg_in_mol, axis = 2), axis = 1))
    #southern ocean
    #get a yearly mean in mol_s_gridcell- takes monthly data, gets out yearly data
    t_yearly = weighted_temporal_mean(cflx_mol_s[:,0:37,:])
    siy = 60*60*24*365 #seconds in year
    pg_in_mol = 12 * 1e-15 #petagrams in a mol
    cflx_pg_yr_so = (np.nansum(np.nansum(t_yearly*siy*pg_in_mol, axis = 2), axis = 1))
    
    t_yearly = weighted_temporal_mean(t_ds['pCO2'])
    glob_mean = masked_average(t_yearly, dim=['y','x'], weights=tmesh['csize'])
    pco2_uatm_yr = (glob_mean.values)
    glob_mean = masked_average(t_yearly[:,0:37,:], dim=['y','x'], weights=tmesh['csize'][0:37,:])
    pco2_uatm_yr_so = (glob_mean.values)  
    
    t_ds.close()
    if verbose:
        print(f'pco2_uatm_yr: {pco2_uatm_yr}')
        print(f'pco2_uatm_yr_so: {pco2_uatm_yr_so}') 
        print(f'cflx_pg_yr: {cflx_pg_yr}')
        print(f'cflx_pg_yr_so: {cflx_pg_yr_so}') 

    ds2 = xr.Dataset(
    {
        "cflx": (["time"], cflx_pg_yr, {"units": "pg/yr"}, {"notes": "co2 flux, whole ocean"}),
        "cflx_so": (["time"], cflx_pg_yr_so, {"units": "pg/yr"}, {"notes": "co2 flux, southern ocean south of -50"}),
        "pco2": (["time"], pco2_uatm_yr, {"units": "uatm"}, {"notes": "surface pco2, whole ocean"}),
        "pco2_so": (["time"], pco2_uatm_yr_so, {"units": "uatm"}, {"notes": "surface pco2, southern ocean south of -50"}),
    },
    coords={
        "yrs": (["time"], yrs)
    },
    attrs=dict(description="model analytics"),
    )
    ds2.to_netcdf(f'{sdir}{fnam}')
        
    return 

In [20]:
sdir = '/gpfs/home/mep22dku/scratch/SOZONE/MODPROC_ROBOT/MultiModelMonitor/OUTPUT/'
fnam = 'test.nc'
get_cflx(1948,1949,tr, baseDir, sdir, fnam)

In [24]:
import netCDF4 as nc
w = nc.Dataset('./OUTPUT/test.nc')
print(w['pco2'][:])

[288.55288154 287.36294615]


/gpfs/home/mep22dku/.conda/envs/swampenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:

w = time.time()
print('ylist')
ylist = make_yearlist(1950, 1951, 'ptrc', tr, ad)
print('open')
t_ds = xr.open_mfdataset(ylist)
# print('done')
# get_ppt(t_ds, True)
# w2 = time.time()
# print(w2-w)

ylist
open


In [27]:
def breakdown_maker(tr, baseDir = '/gpfs/afm/greenocean/software/runs/', \
    fmi = 0, fmx = 3000, bdflag = 'cflx', \
    resDir = '/gpfs/home/mep22dku/scratch/SOZONE/MODPROC_ROBOT/MultiModelMonitor/OUTPUT/',\
    verbose = False):

    '''
    for a given model tr, save some diagnostics
    cflx = 'True': carbon flux (pg/year)'

    '''
    w = time.time()
    #--------------find max and min years in TS, announce intent of where things will end up being
    tmin, tmax = max_min_yrs(tr, baseDir)
    # if we've set forced maxes and mins for start and end years, read them in here to overwrite the above
    if fmi > 0: tmin = fmi;
    if fmx < 3000: tmax = fmx;

    print(f'BEEP BOOP RUNNING SUMMARY PROTOCOL ON MODEL {tr}')
    print(f'analyzing years {tmin}-{tmax}')
    fnam = f'SUMMARY_{bdflag}_{tr}_{tmin}-{tmax}.nc'
    sdir = '/gpfs/home/mep22dku/scratch/SOZONE/MODPROC_ROBOT/CUSTOM_BD/'
    print(f'producing summary stats {fnam}')
    print(f'for storage in {resDir}')
  
    #-------------- cflx extract--------------
    if bdflag == 'cflx':
        get_cflx(tmin,tmax,tr, baseDir, resDir, fnam, verbose)

    w2 = time.time()
    print(f'compute complete, time taken (s): {w2-w}')


    

In [28]:
breakdown_maker('TOM12_TJ_1AS6', fmi = 1948, fmx = 1949)

BEEP BOOP RUNNING SUMMARY PROTOCOL ON MODEL TOM12_TJ_1AS6
analyzing years 1948-1949
producing summary stats SUMMARY_cflx_TOM12_TJ_1AS6_1948-1949.nc
for storage in /gpfs/home/mep22dku/scratch/SOZONE/MODPROC_ROBOT/MultiModelMonitor/OUTPUT/
compute complete, time taken (s): 0.7277531623840332


In [7]:

tr = 'TOM12_DW_WE43'
breakdown_maker(tr, fmi = 1970, fmx = 1980, verbose = True)

BEEP BOOP RUNNING SUMMARY PROTOCOL ON MODEL TOM12_DW_WE43
analyzing years 1970-1980
producing summary stats SUMMARY_TOM12_DW_WE43_1970-1980.nc
for storage in /gpfs/home/mep22dku/scratch/SOZONE/MODPROC_ROBOT/CUSTOM_BD/
cflx_pg_yr: [1.35276295 1.44102403 1.65607843 1.42596247 1.42326032 1.34910156
 1.54146982 1.64860091 1.64003786 1.79489758 2.002411  ]
cflx_pg_yr_so: [0.22623113 0.25114479 0.28343106 0.29050099 0.25336761 0.29148108
 0.26635583 0.33959848 0.33056024 0.31038988 0.36075974]
[51.39015995 51.31719171 50.49036993 50.89664574 52.02343893 51.89767609
 51.75834234 52.20106194 52.27555543 52.01845757 52.41060726]
[2.79440466 2.74959035 2.69423472 2.77326103 2.76384617 2.72810332
 2.7540908  2.91139898 2.78426343 2.78064396 2.89036249]
pco2_uatm_yr: [311.64756818 312.00027835 311.98760018 314.47632715 315.5802953
 316.62854247 316.80189684 317.34972549 319.08367918 320.11868033
 320.91692245]
pco2_uatm_yr_so: [315.23328489 315.63768204 315.50040825 316.60029081 318.74081052
 319.

In [8]:
tl = '1AS6'; tl = 'WE43'

q = glob.glob(f'{sdir}*{tl}*')
print(q[0])

NameError: name 'sdir' is not defined

In [ ]:


w = xr.open_dataset(

In [ ]:
sdir = '/gpfs/home/mep22dku/scratch/SOZONE/MODPROC_ROBOT/CUSTOM_BD/'
tnam = 'SUMMARY_TOM12_TJ_1AS6_1950-1951.nc'
w = xr.open_dataset(f'{sdir}{tnam}')
print(w)
print(w.cflx)

In [ ]:
tmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/regrid/mesh_mask3_6.nc')
tmesh['csize'] = tmesh.tmask[0,0,:,:] * tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:]
t_yearlist = make_yearlist(1950,1951,'diad',tr, ad)
t_ds = xr.open_mfdataset(t_yearlist)
print(t_ds['pCO2'])

#glob_mean.plot()

In [ ]:
np.shape(t_ds['pCO2'].values)

In [ ]:
t_ds['pCO2'][:,0:37,:]